In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import keras
from keras_vggface.vggface import VGGFace
import cv2
import time

2023-08-23 16:34:42.862305: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 16:34:43.795778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
prob_model = load_model("third_model_face.tf")

2023-08-23 16:34:46.550881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 16:34:46.696703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-23 16:34:46.696911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
name = ['bus','elf','petch','unknown']

In [5]:
sample_set = []
for i in range(1,31) :
    sample = cv2.imread(f'test/000{i}.png')
    sample_set.append(sample)


In [6]:
sample_set[24].shape

(132, 130, 3)

In [7]:
def sample_process(sample) :
    sample_reshape = cv2.resize(sample,dsize=(224,224))
    sample = tf.convert_to_tensor(sample_reshape, dtype=tf.float32)
    sample = tf.expand_dims(sample, 0)
    return sample

In [8]:
sample_set_processed = []
for sample in sample_set :
    sample_e = sample_process(sample)
    sample_set_processed.append(sample_e)

In [9]:
sample_set_processed[0].shape

TensorShape([1, 224, 224, 3])

In [10]:
num = 0
for sample in sample_set_processed :
    num+=1
    predictions = prob_model.predict(sample)
    print(predictions[0])
    print(f'num : {num}\tclass : {name[predictions[0].argmax()]} conf : {predictions[0][predictions[0].argmax()] * 100}')

2023-08-23 16:34:55.496379: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-23 16:34:56.652395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 4s 4s/step
[0.00584693 0.97435975 0.00246858 0.01732482]
num : 1	class : elf conf : 97.43597507476807
1/1 [==============================] - 0s 18ms/step
[0.02689626 0.08677001 0.8751167  0.01121705]
num : 2	class : petch conf : 87.51167058944702
1/1 [==============================] - 0s 18ms/step
[0.01033761 0.9405719  0.00748541 0.04160509]
num : 3	class : elf conf : 94.05719041824341
1/1 [==============================] - 0s 18ms/step
[0.98610526 0.00285375 0.00400112 0.00703989]
num : 4	class : bus conf : 98.61052632331848
1/1 [==============================] - 0s 17ms/step
[0.2252686  0.10708395 0.04687657 0.6207709 ]
num : 5	class : unknown conf : 62.07708716392517
1/1 [==============================] - 0s 17ms/step
[0.38509813 0.17151971 0.1483104  0.2950717 ]
num : 6	class : bus conf : 38.50981295108795
1/1 [==============================] - 0s 17ms/step
[0.4191685  0.03250257 0.05230152 0.49602735]
num : 7	class : unknown conf : 49.602735

In [11]:
import cv2, queue, threading, time

In [12]:
class VideoCapture:

  def __init__(self, name):
    self.cap = cv2.VideoCapture(name)
    self.q = queue.Queue()
    t = threading.Thread(target=self._reader)
    t.daemon = True
    t.start()

  def _reader(self):
    while True:
      ret, frame = self.cap.read()
      if not ret:
        break
      if not self.q.empty():
        try:
          self.q.get_nowait()
        except queue.Empty:
          pass
      self.q.put(frame)

  def read(self):
    return self.q.get()
  
  def release(self):
    self.cap.release()


In [14]:
cap = VideoCapture(1)
font = cv2.FONT_HERSHEY_SIMPLEX
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

color = (0, 255, 0)
fontsize = 2

while(True):
    img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
        minSize=(10, 10)
    )
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        predictions = prob_model.predict(sample_process(img[y:y+h,x:x+w]), verbose=0)
        id = predictions[0].argmax()
        confidence = predictions[0][predictions[0].argmax()] * 100
        if (confidence > 80):
            id = name[id]
            cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
            cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
        else:
            id = "unknown"
            cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        
    cv2.imshow('camera',img)
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
print("\n [INFO] Exiting Program and cleanup stuff")
cap.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program and cleanup stuff
